Wie in Übungen

In [2]:
from pandas import DataFrame
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as st
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt

df = pd.read_csv("automob.dat", sep=" ")
df.head()

,STADT.index,AUTO,MPG,KMP4L,STADT
1,1,1,20.3,34.5303,Los Angeles
2,1,1,19.8,33.6798,Los Angeles
3,1,1,21.4,36.4014,Los Angeles
4,1,2,19.5,33.1695,Los Angeles
5,1,2,18.6,31.6386,Los Angeles


In [3]:
fit = ols("KMP4L ~ C(STADT,Sum)*C(AUTO,Sum)", data=df).fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
"C(STADT, Sum)",2.0,19.599255,9.799628,7.438263,2.379745e-03
"C(AUTO, Sum)",4.0,179.741928,44.935482,34.107613,9.163330e-11
"C(STADT, Sum):C(AUTO, Sum)",8.0,244.619694,30.577462,23.209370,7.578447e-11
Residual,30.0,39.523858,1.317462,NaN,NaN


Nun lassen wir das C(..., Sum) weg

In [4]:
fit = ols("KMP4L ~ STADT*AUTO", data=df).fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
STADT,2.0,19.599255,9.799628,1.175204,0.319451
AUTO,1.0,9.621844,9.621844,1.153884,0.289339
STADT:AUTO,2.0,129.055972,64.527986,7.738414,0.001478
Residual,39.0,325.207663,8.338658,NaN,NaN


Wir bekommen komplett andere $p$-Werte. Warum ist dies so? Betrachten wir die Spalte df. Da steht bei C(Auto,Sum) 4 und unten 1. df steht für degree of freedom und ist allgemein 1 weniger als die Anzahl Stufen dieses Faktors, also müsste 4 stehen. Aber im 2. Fall steht 1. Warum? Schauen wir uns die Datenstruktur an:

In [5]:
df.dtypes

STADT.index      int64
AUTO             int64
MPG            float64
KMP4L          float64
STADT           object
dtype: object

Da steht bei Auto int64, das heisst pandas fasst die Einträge als Zahlen auf und nicht als Stufen eines Faktors. Wir müssen also diese Spalte in Objekte umwandeln (wie bei STADT, die aber schon als object vorhanden ist, da die Einträge strings sind). 


In [6]:
df["AUTO"] =  df["AUTO"].astype(object)
df.dtypes

STADT.index      int64
AUTO            object
MPG            float64
KMP4L          float64
STADT           object
dtype: object

In [7]:
df.head()

,STADT.index,AUTO,MPG,KMP4L,STADT
1,1,1,20.3,34.5303,Los Angeles
2,1,1,19.8,33.6798,Los Angeles
3,1,1,21.4,36.4014,Los Angeles
4,1,2,19.5,33.1695,Los Angeles
5,1,2,18.6,31.6386,Los Angeles


Es stehen zwar immer noch Zahlen in der Spalte AUTO, aber die sind als Strings abgespeichert.

Führen wir nun die Varianzanalyse durch:

In [8]:
fit = ols("KMP4L ~ STADT*AUTO", data=df).fit()
anova_lm(fit)

,df,sum_sq,mean_sq,F,PR(>F)
STADT,2.0,19.599255,9.799628,7.438263,2.379745e-03
AUTO,4.0,179.741928,44.935482,34.107613,9.163330e-11
STADT:AUTO,8.0,244.619694,30.577462,23.209370,7.578447e-11
Residual,30.0,39.523858,1.317462,NaN,NaN


Jetzt steht df 4 für AUTO und die $p$-Werte sind dieselben wie mit C(..., Sum). 

Apropos: C() steht für contrast, was man (sehr vereinfacht) als Faktorstufen auffassen kann. 